In [4]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import dotenv
from sqlalchemy import create_engine, text
from typing import List, Dict, Tuple, Optional, Union, Any, Callable
import json

dotenv.load_dotenv()

True

In [5]:
# Start db
def create_conn():
    driver = os.getenv("DB_DRIVER")
    server = os.getenv("DB_SERVER")
    database = os.getenv("DB_NAME")
    trusted_connection = os.getenv("DB_TRUSTED_CONNECTION")

    return create_engine(
        f"mssql+pyodbc://{server}/{database}?trusted_connection={trusted_connection}&driver={driver}"
    )


engine = create_conn()
# Test connection
connection = engine.connect()
res = connection.execute(text("SELECT @@version;")).fetchone()
connection.close()
res[0]

'Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) \n\tOct  8 2022 05:58:25 \n\tCopyright (C) 2022 Microsoft Corporation\n\tDeveloper Edition (64-bit) on Windows 10 Home 10.0 <X64> (Build 19045: ) (Hypervisor)\n'

In [6]:
query = """
SELECT * FROM contact
"""

df = pd.read_sql(query, engine)
df

,crm_contact_id,crm_contact_account,crm_contact_functietitel,crm_persoon_id,crm_contact_status,crm_contact_voka_medewerker
0,E14A8BC8-DCC9-EC11-A7B5-000D3A20A90F,7D15D389-FF2A-E511-93F3-005056B06EC4,CLB arts coordinator,EE4A8BC8-DCC9-EC11-A7B5-000D3A20A90F,Actief,0
1,B548B872-DDC9-EC11-A7B5-000D3A20A90F,E82DAACF-A656-EC11-8F8F-000D3AAAAFCA,Co-Founder,8E77DD5A-7276-EB11-811D-001DD8B72B62,Actief,0
2,9DBB5B1D-DFC9-EC11-A7B5-000D3A20A90F,9EBB5B1D-DFC9-EC11-A7B5-000D3A20A90F,founder of Tablot,F1DF8A23-DFC9-EC11-A7B5-000D3A20A90F,Actief,0
3,1BE002B3-E3C9-EC11-A7B5-000D3A20A90F,A9EA2C80-AC68-E111-B43A-00505680000A,"HR Coördinator WOLL (Welzijn, Ontwikkeling, Le...",1FE002B3-E3C9-EC11-A7B5-000D3A20A90F,Inactief,0
4,805A8764-EAC9-EC11-A7B5-000D3A20A90F,FBC1E08F-B368-E111-B43A-00505680000A,HR Manager,6CE11D6E-4969-E111-B43A-00505680000A,Actief,0
...,...,...,...,...,...,...
1169668,C19AE6AE-585A-4D5D-B8E7-FF92D882BA15,77990022-BA68-E111-B43A-00505680000A,Partner,C12FD79C-EB67-E111-A00F-00505680000A,Actief,0
1169669,AB20792C-9ACE-44DA-B4DB-FFA987F1D608,64036EE0-9E68-E111-B43A-00505680000A,Planningverantwoordelijke,88F84387-4398-4FD7-B579-014471B84267,Inactief,0
1169670,DF4228E0-82BD-4374-BED2-FFB79BC429B4,52D13234-0669-E111-B43A-00505680000A,Team Leader Gemeentelijke Infra,04E69AD1-9FA8-49AE-AF0D-7966D1A8DE72,Actief,0
1169671,8E3FF0E2-14CD-483D-AC00-FFE2CFB5E136,4E996FAB-0869-E111-B43A-00505680000A,woordvoerder bpost,E5FA1050-349C-4D35-8C3E-5BD42C72D0BD,Inactief,0


In [8]:
# Import the data

with open(os.getenv("IMPORT_DATA_PATH"), "r") as f:
    import_data: List[Dict] = json.load(f)
import_data

[{'file_name': 'teams.csv',
  'table_name': 'teams',
  'columns': {'XLS_Teams_Team_code_selecteer_uit_lijst_': 'xls_teams_team_code_selecteer_uit_lijst',
   'XLS_Teams_Activiteit_boeking_naam_ter_info_': 'XLS_Teams_Activiteit_boeking_naam_ter_info'},
  'refactor': {}},
 {'file_name': 'Afspraak alle.csv',
  'table_name': 'afspraak_alle',
  'columns': {'crm_Afspraak_ALLE_Afspraak': 'crm_afspraak_id'},
  'refactor': {}},
 {'file_name': 'Activiteit vereist contact.csv',
  'table_name': 'activiteit_vereist_contact',
  'columns': {'crm_activiteitvereistcontact_activityid': 'crm_afspraak_id',
   'crm_activiteitvereistcontact_reqattendee': 'crm_activiteitvereistcontact_reqattendee'},
  'refactor': {}},
 {'file_name': 'Afspraak betreft account_cleaned.csv',
  'table_name': 'afspraak_betreft_account',
  'columns': {'crm_afspraak_betreft_account_afspraak': 'crm_afspraak_id',
   'crm_afspraak_betreft_account_thema': 'crm_afspraak_betreft_account_thema',
   'crm_afspraak_betreft_account_subthema': 

In [9]:
# Get the table_name afspraak_betreft_contact
table_name = "afspraak_betreft_contact"

data = [x for x in import_data if x["table_name"] == table_name][0]

data

{'file_name': 'Afspraak betreft contact_cleaned.csv',
 'table_name': 'afspraak_betreft_contact',
 'columns': {'crm_afspraak_betreft_contactfiche_betreft_id': 'crm_afspraak_id',
  'crm_afspraak_betreft_contactfiche_thema': 'crm_afspraak_betreft_contactfiche_thema',
  'crm_afspraak_betreft_contactfiche_subthema': 'crm_afspraak_betreft_contactfiche_subthema',
  'crm_afspraak_betreft_contactfiche_onderwerp': 'crm_afspraak_betreft_contactfiche_onderwerp',
  'crm_afspraak_betreft_contactfiche_afspraak': 'crm_contact_id',
  'crm_afspraak_betreft_contactfiche_eindtijd': 'crm_afspraak_betreft_contactfiche_eindtijd',
  'crm_afspraak_betreft_contactfiche_keyphrases': 'crm_afspraak_betreft_contactfiche_keyphrases'},
 'refactor': {'crm_afspraak_betreft_contactfiche_eindtijd': 'to_date'}}

In [10]:
# Load the data into a dataframe
df2 = pd.read_csv(os.path.join(os.getenv("IMPORT_CSV_PATH"), data["file_name"]))
df2

,crm_Afspraak_BETREFT_CONTACTFICHE_Afspraak,crm_Afspraak_BETREFT_CONTACTFICHE_Thema,crm_Afspraak_BETREFT_CONTACTFICHE_Subthema,crm_Afspraak_BETREFT_CONTACTFICHE_Onderwerp,crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id,crm_Afspraak_BETREFT_CONTACTFICHE_Eindtijd,crm_Afspraak_BETREFT_CONTACTFICHE_KeyPhrases
0,00082DAA-24BF-ED11-83FF-6045BD8952CE,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2023,33F751E0-D26B-E111-B43A-00505680000A,18-4-2023,"[NAME] [NAME] [NAME] , grootste visverwerkende..."
1,00082DAA-24BF-ED11-83FF-6045BD8952CE,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2023,33F751E0-D26B-E111-B43A-00505680000A,18-4-2023,"[NAME] [NAME] [NAME] , grootste visverwerkende..."
2,00109C77-4D09-EB11-8114-001DD8B72B61,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2020,B9B048FF-A868-E111-B43A-00505680000A,16-10-2020,"china en india , india en china , verschillend..."
3,0011A5D9-CC9A-EC11-B400-0022488005A7,Financieel,Financiering,OV-Kennismaking inzake funding,C9D63042-CC9A-EC11-B400-0022488005A7,28-2-2022,"interesse , ervaringen , ondernemers , investe..."
4,0016532E-14E3-EC11-BB3C-000D3ABCA38B,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022,3E4CE37B-D069-E111-B43A-00505680000A,24-6-2022,"verwerkende fabriek , nodeloze oproepingen , g..."
...,...,...,...,...,...,...,...
8590,FFD83E86-6F48-ED11-BBA1-6045BD895D85,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022,91AF5A78-336A-E111-B43A-00505680000A,27-10-2022,"[NAME] [NAME] [NAME] , huidige economische con..."
8591,FFF09F47-8BC5-E911-8104-001DD8B72B61,Lidmaatschap,Retentie (Lidmaatschap),Voka-Ledenbezoek-2019,783E0F2C-AC68-E111-B43A-00505680000A,12-9-2019,"verschillende industrieën , belgië en luxembur..."
8592,FFF3E77B-8545-EE11-BE6E-6045BD895BFB,"Digitalisering, IT & Technologie",Big Data,Ov-update 23-digitalisering en innovatie,05C7BB1A-B567-E111-A00F-00505680000A,28-8-2023,"r&d manager , maleisie en turkije , data archi..."
8593,FFF6064B-8616-EE11-8F6D-6045BD8952CE,Plato,Prospectie,OV-PLATO-intake,C23997B9-45BD-4854-983C-05E767BE35D1,13-7-2023,"industriële steigerbouw , eigen aandeel , posi..."


In [13]:
# Merge the dataframes on the columns crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id (df2) & crm_contact_id (df)

df3 = pd.merge(
    df2,
    df,
    how="inner",
    left_on="crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id",
    right_on="crm_contact_id",
)

df3

,crm_Afspraak_BETREFT_CONTACTFICHE_Afspraak,crm_Afspraak_BETREFT_CONTACTFICHE_Thema,crm_Afspraak_BETREFT_CONTACTFICHE_Subthema,crm_Afspraak_BETREFT_CONTACTFICHE_Onderwerp,crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id,crm_Afspraak_BETREFT_CONTACTFICHE_Eindtijd,crm_Afspraak_BETREFT_CONTACTFICHE_KeyPhrases,crm_contact_id,crm_contact_account,crm_contact_functietitel,crm_persoon_id,crm_contact_status,crm_contact_voka_medewerker
0,00082DAA-24BF-ED11-83FF-6045BD8952CE,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2023,33F751E0-D26B-E111-B43A-00505680000A,18-4-2023,"[NAME] [NAME] [NAME] , grootste visverwerkende...",33F751E0-D26B-E111-B43A-00505680000A,513DC17D-A968-E111-B43A-00505680000A,Zaakvoerder; ceo,F7C20B52-4169-E111-B43A-00505680000A,Actief,0
1,00082DAA-24BF-ED11-83FF-6045BD8952CE,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2023,33F751E0-D26B-E111-B43A-00505680000A,18-4-2023,"[NAME] [NAME] [NAME] , grootste visverwerkende...",33F751E0-D26B-E111-B43A-00505680000A,513DC17D-A968-E111-B43A-00505680000A,Zaakvoerder; ceo,F7C20B52-4169-E111-B43A-00505680000A,Actief,0
2,0011A5D9-CC9A-EC11-B400-0022488005A7,Financieel,Financiering,OV-Kennismaking inzake funding,C9D63042-CC9A-EC11-B400-0022488005A7,28-2-2022,"interesse , ervaringen , ondernemers , investe...",C9D63042-CC9A-EC11-B400-0022488005A7,C63C4F8E-CF12-EB11-8114-001DD8B72B61,CEO,DD99402F-8553-E311-BBFD-005056B06EB4,Actief,0
3,0016532E-14E3-EC11-BB3C-000D3ABCA38B,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022,3E4CE37B-D069-E111-B43A-00505680000A,24-6-2022,"verwerkende fabriek , nodeloze oproepingen , g...",3E4CE37B-D069-E111-B43A-00505680000A,2BA29C15-9368-E111-B43A-00505680000A,CEO,AB16F655-AB67-E111-A00F-00505680000A,Actief,0
4,003760C1-13B4-EC11-983F-00224883C880,Lidmaatschap,Retentie (Lidmaatschap),OV Ledengesprek 2022,622D5DC9-416B-E111-B43A-00505680000A,21-4-2022,"aal arbeider vti waregem , [NAME] [NAME] [NAME...",622D5DC9-416B-E111-B43A-00505680000A,A06B2F59-C368-E111-B43A-00505680000A,Bedrijfsleider,E2C0011C-1468-E111-A00F-00505680000A,Actief,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2865,FF676F29-BB38-ED11-9DB1-6045BD8952CE,Welt 2.0,Dieptegesprek met Full Scan,Welt 2.0. Dieptegesprek met Full scan,EDAB7126-0CFD-EC11-82E5-000D3A3A3004,19-9-2022,NaN,EDAB7126-0CFD-EC11-82E5-000D3A3A3004,816E1535-31B7-E111-A45C-00505680000A,HR BP,F1AB7126-0CFD-EC11-82E5-000D3A3A3004,Actief,0
2866,FF76BB42-B451-EC11-8C62-000D3ABFCBA8,Lidmaatschap,Prospectie (Lidmaatschap),Kennismaking ACB en Voka Oost-Vlaanderen,8930DBFD-FAD5-E811-80F7-001DD8B72B61,7-12-2021,"en wim huel , ln sales directors , ln producti...",8930DBFD-FAD5-E811-80F7-001DD8B72B61,C16E8BCF-9368-E111-B43A-00505680000A,sales director,FDF4E1F3-FAD5-E811-80F7-001DD8B72B61,Actief,0
2867,FF858E81-1775-EC11-8943-000D3ABB8FB4,Lidmaatschap,Prospectie (Lidmaatschap),Kennismaking Cos & Voka OVL,A2C170D6-90BD-E411-AC4D-005056B06EC4,25-1-2022,"[NAME] [NAME] [NAME] , eerder gesprek , drukke...",A2C170D6-90BD-E411-AC4D-005056B06EC4,D1F1618F-2769-E111-B43A-00505680000A,Bedrijfsleider,323728BF-90BD-E411-AC4D-005056B06EC4,Actief,0
2868,FFB8754D-B0BD-ED11-83FF-6045BD895D85,Lidmaatschap,Retentie (Lidmaatschap),OV-Ledenbezoek 2023,BFC4C705-ECC1-E111-A45C-00505680000A,24-4-2023,"[NAME] of anja , winnewald dejonckheere , nieu...",BFC4C705-ECC1-E111-A45C-00505680000A,F211D444-0A69-E111-B43A-00505680000A,?common sense consultant,8C1BF56A-C1C1-E111-A45C-00505680000A,Actief,0


In [14]:
# Merge the dataframes on the columns crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id (df2) & crm_contact_id (df)

df4 = pd.merge(
    df2,
    df,
    how="inner",
    left_on="crm_Afspraak_BETREFT_CONTACTFICHE_Afspraak",
    right_on="crm_contact_id",
)

df4

,crm_Afspraak_BETREFT_CONTACTFICHE_Afspraak,crm_Afspraak_BETREFT_CONTACTFICHE_Thema,crm_Afspraak_BETREFT_CONTACTFICHE_Subthema,crm_Afspraak_BETREFT_CONTACTFICHE_Onderwerp,crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id,crm_Afspraak_BETREFT_CONTACTFICHE_Eindtijd,crm_Afspraak_BETREFT_CONTACTFICHE_KeyPhrases,crm_contact_id,crm_contact_account,crm_contact_functietitel,crm_persoon_id,crm_contact_status,crm_contact_voka_medewerker
